# merge dataframe

- https://datacarpentry.org/python-ecology-lesson/05-merging-data/index.html
- https://datacarpentry.org/python-socialsci/11-joins/index.html

In [ ]:
from io import BytesIO
from zipfile import ZipFile
import pandas
import requests
import itertools

#url = 'https://www.cms.gov/files/zip/monthly-enrollment-plan-december-2020.zip'

url = ['https://www.cms.gov/files/zip/monthly-enrollment-plan-october-2020.zip',
       'https://www.cms.gov/files/zip/monthly-enrollment-plan-november-2020.zip',
       'https://www.cms.gov/files/zip/monthly-enrollment-plan-december-2020.zip',
       'https://www.cms.gov/files/zip/monthly-enrollment-plan-january-2021.zip']




date = ['2020-10-31', 
        '2020-11-30', 
        '2020-12-31', 
        '2021-01-31']

file = ['Monthly_Report_By_Plan_2020_10/Monthly_Report_By_Plan_2020_10.csv', 
        'Monthly_Report_By_Plan_2020_11/Monthly_Report_By_Plan_2020_11.csv', 
        'Monthly_Report_By_Plan_2020_12/Monthly_Report_By_Plan_2020_12.csv', 
        'Monthly_Report_By_Plan_2021_01/Monthly_Report_By_Plan_2021_01.csv']


new_columns = ['Month', 'Name', 'Enrollment']

df6 = pd.DataFrame(columns=new_columns)

for (a, b, c) in zip(url, date, file):
    
    filename = requests.get(a).content
    zf = ZipFile( BytesIO(filename), 'r' )

    for item in zf.namelist():
        print("File in zip: "+ item)

    match = [s for s in zf.namelist() if c in s][0]
    #df = pandas.read_csv( zf.open(match), encoding='utf-8', error_bad_lines=False)

    df = pd.read_csv(zf.open(match), encoding='latin1')

    #print(df)

    #df.head(5)

    # rename

    df1 = df.rename(columns={'Contract Number': 'Contract'})
    df1 = df1.rename(columns={'Plan Name': 'Name'})
    
    # Contract = H6988
    df2 = df1[(df1["Contract"] == 'H6988')]

    #df2

    # select columns

    df3 = df2[['Name', 'Enrollment']]

    # change data
    df3 = df3.replace(['Centers Plan for Medicare Advantage Care (HMO)'],'MAPD')
    df3 = df3.replace(['Centers Plan for Dual Coverage Care (HMO D-SNP)'],'D-SNP')
    df3 = df3.replace(['Centers Plan for Nursing Home Care (HMO I-SNP)'],'I-SNP')
    df3 = df3.replace(['Centers Plan for Medicaid Advantage Plus (HMO D-SNP)'],'MAP')
    df4 = df3.replace(['Centers Plan for Employer Groups (HMO)'],'Employer')

    #df4

    # drop last row

    df5 = df4[df4.Name != 'Employer']
    df5 = df5[df5.Name != 'Centers Plan for Medicaid Advantage (HMO D-SNP)']

    # add month
    df5['Month'] = b

    # change order
    df5 = df5[['Month', 'Name', 'Enrollment']]
    
    df6 = df6.append(df5, ignore_index=True)

print (df6)

df6.to_excel(r'4.xlsx', index = False)

# dataframe to pivot
df7 = pd.pivot_table(df6, index=['Month'], columns=['Name'], values=['Enrollment'], aggfunc='first')

# pivot to dataframe
df8 = pd.DataFrame(df7.to_records())

# Change columns name
df9 = df8.rename(columns={"('Enrollment', 'D-SNP')": "D-SNP", "('Enrollment', 'I-SNP')": "I-SNP", "('Enrollment', 'MAP')": "MAP", "('Enrollment', 'MAPD')": "MAPD"})
df9

#df9.dtypes